In [1]:
%pip install census
%pip install us


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import modules
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from census import Census
from us import states
import os
from tqdm import tqdm

In [3]:
#read in analysis data set to get unique list of all zipcodes
analysis_df = pd.read_csv('/Users/skareti/Desktop/Independent Study/NYTRestaurantReviews/data/AnalysisDataset.csv', index_col=0)
#also read in modeling df so that we can join all cencus data at the end
modeling_df = pd.read_csv('/Users/skareti/Desktop/Independent Study/NYTRestaurantReviews/data/modeling_df.csv')

In [4]:
analysis_df.head()

,Restaurant_name,Author,Publishing_Date,Rating,Price,Cuisine,Neighborhood,CriticsPick,Description,Rank,...,nta2020,ntaname,ntaabbrev,ntatype,cdta2020,cdtaname,shape_leng,shape_area,isNYC,Status
2,'Wichcraft,PETER MEEHAN,"Sept. 7, 2005",NaN,$,"American, Sandwiches",Midtown South,0,The sandwiches are good at this Tom Colicchio-...,0.0,...,MN0603,Murray Hill-Kips Bay,MryHl_KpBy,0.0,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,25692.742059,1.577115e+07,1,Open
3,12 Chairs,ERIC ASIMOV,"Oct. 18, 1996",NaN,$$,Russian,SoHo,0,"It offers Middle Eastern dishes, American sand...",0.0,...,MN0201,SoHo-Little Italy-Hudson Square,SoHo,0.0,MN02,MN02 Greenwich Village-SoHo (CD 2 Equivalent),20252.906482,1.291676e+07,1,Open
4,15 East,FRANK BRUNI,"July 11, 2007",2 star,$$$$,"Japanese, Sushi",Union Square,0,15 East stands out in a crowded marketplace of...,0.0,...,MN0501,Midtown South-Flatiron-Union Square,MdtwnSth,0.0,MN05,MN05 Midtown-Flatiron-Union Square (CD 5 Appro...,18800.765628,1.487811e+07,1,Open
5,188 Bakery Cuchifritos,PETE WELLS,"April 2, 2024",NaN,$,"Puerto Rican, Dominican",Fordham Heights,0,NaN,86.0,...,BX0503,Fordham Heights,FrdhmHts,0.0,BX05,BX05 Morris Heights-Mount Hope (CD 5 Approxima...,12917.279642,7.379040e+06,1,Open
6,2nd City,LIGAYA MISHAN,"Sept. 15, 2016",NaN,$$,Philippine,West Village,0,"At this Filipino-leaning taqueria, the chef Jo...",0.0,...,MN0203,West Village,WstVlg,0.0,MN02,MN02 Greenwich Village-SoHo (CD 2 Equivalent),23882.339337,1.441774e+07,1,Open


In [5]:
import geopy
import pandas as pd

def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    return location.raw['address']['postcode']

geolocator = geopy.Nominatim(user_agent='nytRestaurantsReviews')
analysis_df['zipcode'] = analysis_df.apply(get_zipcode, axis = 1, geolocator=geolocator, lat_field = 'Latitude', lon_field = 'Longitude')


In [6]:
zipcodes = analysis_df['zipcode'].unique().tolist()

In [7]:
c = Census("f06e9319647ca5f1a2410e5399427b5874d63b36", year = 2022)

In [8]:
# Initialize an empty list to hold data for each ZIP code
census_data_list = []

# Loop through the list of ZIP codes
for zipcode in tqdm(zipcodes):
    # Pull data for each ZIP code using state_zipcode method
    census_data_zip = c.acs5.state_zipcode(
        fields=(
            'NAME', 'B19013_001E', 'B01003_001E', 'C16001_002E',
            'C16001_003E', 'C16001_004E', 'C16001_005E', 'C16001_006E',
            'B15003_001E', 'B15003_022E', 'B15003_023E', 'B15003_024E'
        ),
        state_fips=states.NY.fips,
        zcta=zipcode  # Pass each ZIP code individually
    )
    # Append the result to the list
    census_data_list.extend(census_data_zip)

# Convert the list of results to a DataFrame
census_data = pd.DataFrame(census_data_list)

# Display the first few rows of the DataFrame
print(census_data.head())


100%|██████████| 119/119 [02:17<00:00,  1.15s/it]


          NAME  B19013_001E  B01003_001E  C16001_002E  C16001_003E  \
0  ZCTA5 10016     145864.0      54369.0      36846.0       5240.0   
1  ZCTA5 10012     126890.0      21216.0      15813.0       1222.0   
2  ZCTA5 10003     152863.0      53877.0      40948.0       3278.0   
3  ZCTA5 10468      43985.0      81397.0      15410.0      53330.0   
4  ZCTA5 10014     147267.0      29461.0      24073.0       1876.0   

   C16001_004E  C16001_005E  C16001_006E  B15003_001E  B15003_022E  \
0       4180.0       1060.0        871.0      43500.0      18102.0   
1        855.0        367.0        450.0      16809.0       8292.0   
2       2489.0        789.0       1757.0      37093.0      16265.0   
3      25403.0      27927.0        773.0      52353.0       5028.0   
4       1699.0        177.0        726.0      25527.0      12251.0   

   B15003_023E  B15003_024E zip code tabulation area  
0      10877.0       4803.0                    10016  
1       3089.0       1217.0                    1

In [12]:
analysis_df.to_csv('/Users/skareti/Desktop/Independent Study/NYTRestaurantReviews/data/censusdata.csv')

In [ ]:
analysis_df['Description']